testing Torch backend:
* circuit : oscillator, transmon, fluxonium

In [58]:
from DiSuQ.Torch import models
from torch import tensor
from numpy import arange,log10,sqrt,diff
from DiSuQ.utils import plotCompare

### LC oscillator

In [25]:
El = .112
Ec = .0054
circuit = models.oscillatorLC([50],El=El,Ec=Ec,sparse=False)

In [26]:
circuit.circuitState()

{'L': 0.11199995875358582, 'C': 0.005399998277425766}

In [27]:
H_LC = circuit.oscillatorHamiltonianLC()
oscillator = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [28]:
sqrt(8*El*Ec)

0.06955860838170931

In [29]:
H_LC = circuit.chargeHamiltonianLC()
charge = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [30]:
plotCompare(arange(len(oscillator)),{'oscillator':oscillator,'charge':charge},'Energy Spectrum','levels','E(GHz)')

## comparison : Capacitive :: Inductive

In [144]:
from DiSuQ.Torch import components
from DiSuQ.Torch import circuit
from DiSuQ.Torch import dense

In [105]:
C(0,1,Ec=10).capacitance(), L(0,1,El=.025).inductance()

(tensor(0.0500, grad_fn=<MulBackward0>),
 tensor(1.0132, grad_fn=<MulBackward0>))

In [145]:
components.indEnergy(.025)

1.0132118364233778

In [146]:
components.capEnergy(4050)

0.0001234567901234568

In [141]:
cap = circuit.hamiltonianEnergy(dense.basisQq(100)@dense.basisQq(100)/2/4050)

In [142]:
ind = circuit.hamiltonianEnergy(dense.basisFf(100)@dense.basisFf(100)/2/.025)

In [140]:
cap/ind

tensor([   nan, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998, 0.0998,
        0.0998, 0.0998, 0.0998, 0.0998, 

In [52]:
El = 1e-5
Ec = 100
circuit = models.oscillatorLC([500],El=El,Ec=Ec,sparse=False)

In [53]:
H_LC = circuit.chargeHamiltonianLC()
charge = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [54]:
charge[:3]

array([1.6455806e-05, 4.0000003e+02, 4.0000003e+02], dtype=float32)

In [55]:
Ec = 1e-5
El = 1000
circuit = models.oscillatorLC([500],El=El,Ec=Ec,sparse=False)

In [56]:
H_LC = circuit.fluxHamiltonianLC()
flux = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [84]:
N = 10
plotCompare(arange(N),{'Ec':log10(cap),'El':log10(ind)})

/tmp/ipykernel_944248/3138489726.py:2: RuntimeWarning:

divide by zero encountered in log10



### Transmon

In [6]:
Ej,Ec = 20,.3

In [7]:
basis = {'O':[],'I':[],'J':[100]}
circuit = models.transmon(basis,Ej,Ec,sparse=False)
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson
kerman = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [8]:
circuit = models.transmon([100],Ej,Ec,sparse=False)
H_LC=circuit.chargeHamiltonianLC()
H_J=circuit.josephsonCharge
charge = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [9]:
kerman = kerman.detach().numpy()
charge = charge.detach().numpy()
plotCompare(arange(len(kerman)),{'kerman':kerman,'charge':charge},'Energy Spectrum','levels','E(GHz)')

## Fluxonium

In [73]:
basis = {'O':[1000],'I':[],'J':[]}
circuit = models.fluxonium(basis,Ej=100,sparse=False)

In [74]:
flux_profile = tensor(arange(0,1,.01))
flux_profile = [[flux] for flux in flux_profile]
flux_point = ('I')

In [75]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson
E0,EI = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=1)
E0,EII = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=2)

In [76]:
EI = tensor(EI).detach().numpy()
EII = tensor(EII).detach().numpy()

In [77]:
plotCompare(arange(0,1,.01),{'Ist Excitation':EI,'IInd Excitation':EII,'Anharmonicity':EII-EI-EI},
            'Fluxonium-Excitation','external flux','E(GHz)')

In [28]:
log10 = lambda x:x

In [29]:
plot = {'Qq':[],'Fq':[],'Qf':[],'Ff':[],'ratio':[]}
for size in arange(2,500):
    plot['Qq'].append(log10(circuit.backend.basisQq(size).norm().item()))
    plot['Fq'].append(log10(circuit.backend.basisFq(size).norm().item()))
    #plot['Qf'].append(circuit.backend.basisQf(size).norm().item())
    #plot['Ff'].append(circuit.backend.basisFf(size).norm().item())
    plot['ratio'].append(log10(circuit.backend.basisQq(size).norm().item()/circuit.backend.basisFq(size).norm().item()))

In [30]:
plotCompare(arange(2,500),plot,'Operator Norm','n',size=(600,800))